In [23]:
[1,2,3] + [4,5,6]
np.array([1,2,3]) * 20

array([20, 40, 60])

In [19]:
import numpy as np

np.random.beta(5,15,size=200).mean()
np.random.gamma(0.75, 1, size=3400).mean()
SIZE = 2000
arr = np.random.dirichlet([6, 5, 3, 3, 3], size=SIZE)
print(arr)
np.sum(arr, axis=0)/SIZE


[[0.39634635 0.19833763 0.2723631  0.07806727 0.05488565]
 [0.2299572  0.31253032 0.105982   0.18625216 0.16527832]
 [0.24800955 0.2298572  0.23156621 0.10262082 0.18794623]
 ...
 [0.28994514 0.20612512 0.13641767 0.28581112 0.08170094]
 [0.23612075 0.10008648 0.12347496 0.40379759 0.13652022]
 [0.14334022 0.44006096 0.11062517 0.09803004 0.20794362]]


array([0.29569377, 0.25204793, 0.15236331, 0.15175327, 0.14814172])

In [15]:
import numpy as np
import math
n_arms = np.full((5,4), 0, dtype=float)
n_arms[2][1] = 4
for i in range(0,4):
    n_arms[3][i] = 21-i
# np.divide(np.full((5,4), 0), n_arms)
t=627
a = np.full((5,4), 2*math.log(t), dtype=float)
np.divide(a, n_arms, out=np.full_like(a, np.inf, dtype=float), where=n_arms!=0)
mat = np.sqrt(np.divide(a, n_arms, out=np.full_like(a, np.inf, dtype=float), where=n_arms!=0))
print(mat)
np.argmax(mat, axis=1)
mat[[0,1,3], :][:, [0,2]]


[[       inf        inf        inf        inf]
 [       inf        inf        inf        inf]
 [       inf 1.79456771        inf        inf]
 [0.78321356 0.80255508 0.82340414 0.84596733]
 [       inf        inf        inf        inf]]


array([[       inf,        inf],
       [       inf,        inf],
       [0.78321356, 0.82340414]])

In [16]:
from Model.InteractionNode import InteractionNode

In [17]:
print("===")
print("Product={}, PriceLevel={}, Bought={}, #Units={}".format(2,3,"Si",45))


l21 = InteractionNode(3, 4, 0, 0, [])
l22 = InteractionNode(2, 1, 0, 0, [])
l1 = InteractionNode(1, 2, 1, 34, [l21,l22])

l1.printInteractions()

===
Product=2, PriceLevel=3, Bought=Si, #Units=45


TypeError: InteractionNode.__init__() missing 4 required positional arguments: 'firstSlot', 'secondSlot', 'sec1Opened', and 'sec2Opened'

In [9]:
import numpy as np

def softmax(x):
    y = np.exp(x - np.max(x))
    f_x = y / np.sum(np.exp(x))
    return f_x

softmax(np.array([30,20,20,15,15]))

arr = np.array([30,20,20,15,15])
arr / arr.sum()

array([0.3 , 0.2 , 0.2 , 0.15, 0.15])

In [ ]:
import pprint
import sys
import unittest
from Model.GraphProbabilities import GraphProbabilities
from Model.Product import Product
from Model.UserClass import UserClass
from Model.constants import PROBABILITY_MATRIX, SECONDARY_PRODUCTS

id = 0
conversionRate = [
    [1, 0.3, 0.9, 0.2],
    [1, .2, .56, .12],
    [1, .34, .57, .88],
    [1, .2, .56, .12],
    [1, 0.9, 0.17, 0.96]
]

PROBABILITY_MATRIX_3 = [[0, 1, 1, 0, 0],
                      [0.8, 0, 0, 0.9, 0],
                      [0, 1, 0, 0.7, 0],
                      [0.9, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0]]
clickProbability = GraphProbabilities(PROBABILITY_MATRIX_3)
alphas = [1,0,0,0,0]

n_bought_mean = 2
n_bought_variance = 1
n_user_mean = 10
n_user_variance = 1

productList = [Product(int(key), SECONDARY_PRODUCTS[key]) for key in SECONDARY_PRODUCTS]

Lambda = 1

userClass = UserClass(id=id, conversionRate=conversionRate, clickProbability=clickProbability, alphas=alphas,
                        Lambda=Lambda, n_bought_variance=n_bought_variance, n_bought_mean=n_bought_mean,
                        n_user_mean=n_user_mean, n_user_variance=n_user_variance, productList=productList,
                        features_generator=[{"name":"Over 18","probability":0.6},{"name":"Male","probability":0.9}],
                        debug=False)

# pp = pprint.PrettyPrinter(indent=4)
# pp.pprint(userClass.__dict__)

currentPrice = [0, 0, 0, 0, 0]
userClass.setCurrentPrice(currentPrice)

userInteractions = userClass.generateEpisode()
userInteractions.printInteractions()

In [2]:
np.ceil([.65,1.34,2.43,.21,.001])

array([1., 2., 3., 1., 1.])

In [ ]:
def yummy(time, env):
    print("Time: {}".format(time))
    print("Env: {}".format(env.Lambda))
from Environment import Environment
e = Environment(0.3, [userClass])
e.addTimeListener(2, yummy)
e.addTimeListener(2, yummy)
e.round()

In [1]:
from Model.GraphProbabilities import GraphProbabilities
from Model.Product import Product
from Model.UserClass import UserClass
from Model.Evaluator.GraphEvaluator import GraphEvaluator
import numpy as np

CLICK_PROB = [[0, .8, .7, 0, 0],
            [0, 0, 0, .9, .8],
            [0, 1, 0, .75, 0],
            [.9, 0, 0, 0, .82],
            [0, 0, .87, .92, 0]]
L = 0.5
LAMBDA_MATRIX = [[0, 1, L, 0, 0],
                [0, 0, 0, L, 1],
                [0, L, 0, 1, 0],
                [L, 0, 0, 0, 1],
                [0, 0, 1, L, 0]]
conversionRate = [
    [0.9, 0.9, 0.9, 0.9, 0.9],
    [.62, .62, .62, .62, .62],
    [.82, .82, .82, .82, .82],
    [.56, .56, .56, .56, .56],
    [.92, 0.92, 0.92, 0.92, .92]
]
convRates = [.9, .62, .82, .56, .92]
alphas = [0.2, 0.1, 0.3, 0.1, 0.3]
SECONDARY_PRODUCTS = {'0': [1, 2], '1': [4, 3], '2': [3, 1], '3': [4, 0], '4': [2, 3]}
margins = [12,7,9,19,5]
units_mean = [0.75, 1.25, 1.6, 2.1, 0.94]

w_matrix = GraphProbabilities(CLICK_PROB)
lambda_matrix = GraphProbabilities(LAMBDA_MATRIX)
conv_matrix = GraphProbabilities(conversionRate)
y_matrix = w_matrix.multiplyElementWise(lambda_matrix)
y_matrix = y_matrix.multiplyElementWise(conv_matrix)

productList = [Product(int(key), SECONDARY_PRODUCTS[key]) for key in SECONDARY_PRODUCTS]
eval = GraphEvaluator(products_list=productList, click_prob_matrix=CLICK_PROB, lambda_prob=0.7, conversion_rates=convRates,
                    alphas=alphas, margins=margins, units_mean=units_mean, verbose=True)
print("Expected profit over all interactions: {}".format(eval.computeMargin()))

profitSingle = np.multiply(np.array(alphas),np.multiply(np.array(margins),np.array(convRates))).sum()
print("Expected profit over single products: {}".format(profitSingle))

[[0.0, 0.6480000000000001, 0.19845, 0.0, 0.0], [0.0, 0.0, 0.0, 0.121086, 0.30752], [0.0, 0.23533999999999997, 0.0, 0.5043, 0.0], [0.09878400000000002, 0.0, 0.0, 0.0, 0.25715200000000005], [0.0, 0.0, 0.736368, 0.2725408, 0.0]]
Reached nodes in 1-step: 1; 2; 
Reached nodes in 2-step: 1; 3; 4; 
Reached nodes in 3-step: 2; 3; 4; 
Reached nodes in 4-step: 2; 3; 4; 
Expected value margin for product 0 as starting is 43 

Reached nodes in 1-step: 3; 4; 
Reached nodes in 2-step: 0; 2; 3; 4; 
Reached nodes in 3-step: 0; 2; 3; 
Reached nodes in 4-step: 0; 2; 
Expected value margin for product 1 as starting is 46 

Reached nodes in 1-step: 1; 3; 
Reached nodes in 2-step: 0; 3; 4; 
Reached nodes in 3-step: 0; 1; 3; 4; 
Reached nodes in 4-step: 0; 4; 
Expected value margin for product 2 as starting is 40 

Reached nodes in 1-step: 0; 4; 
Reached nodes in 2-step: 1; 2; 
Reached nodes in 3-step: 1; 4; 
Reached nodes in 4-step: 2; 4; 
Expected value margin for product 3 as starting is 29 

Reached nod

In [21]:
from Model.GraphProbabilities import GraphProbabilities
from Model.Evaluator.GraphEvaluator import GraphEvaluator
from Model.Product import Product
from Model.UserClass import UserClass
import numpy as np

CLICK_PROB = [[0, .8, .7, 0, 0],
            [0, 0, 0, .9, .8],
            [0, 1, 0, .75, 0],
            [.9, 0, 0, 0, .82],
            [0, 0, .87, .92, 0]]
L = 0.5
convRates = [.9, .62, .82, .56, .92]
alphas = [0.2, 0.1, 0.3, 0.1, 0.3]
SECONDARY_PRODUCTS = {'0': [1, 2], '1': [4, 3], '2': [3, 1], '3': [4, 0], '4': [2, 3]}
margins = [12,7,9,19,5]
units_mean = [0.75, 1.25, 1.6, 2.1, 0.94]

productList = [Product(int(key), SECONDARY_PRODUCTS[key]) for key in SECONDARY_PRODUCTS]
eval = GraphEvaluator(products_list=productList, click_prob_matrix=CLICK_PROB, lambda_prob=0.7, conversion_rates=convRates,
                    alphas=alphas, margins=margins, units_mean=units_mean, verbose=False)
print("Expected profit over all interactions: {}".format(eval.computeMargin()))

profitSingle = np.multiply(np.array(alphas),np.multiply(np.array(margins),np.array(convRates))).sum()
print("Expected profit over single products: {}".format(profitSingle))

TypeError: object of type 'float' has no len()

In [27]:
from Learner.GreedyLearner import *

conversionRateLevels = [
    [1, 0.8, 0.6, 0.4],
    [1, 0.8, 0.6, 0.4],
    [1, 0.8, 0.6, 0.4],
    [1, 0.8, 0.6, 0.4],
    [1, 0.8, 0.6, 0.4]
]

marginsPerPrice = [[1, 4, 23, 23],  # Product 1
                   [1, 22, 14, 32],  # Product 2
                   [1, 7, 3, 4],  # Product 3
                   [1, 8, 12, 9],  # Product 4
                   [1, 2, 50, 100]  # Product 5
                   ]

units_mean = [0.75, 1.25, 1.6, 2.1, 0.94]

gLearner = GreedyLearner()
optimal_arm = []

for i in range(0, 300):
    pulledArm = gLearner.pull_arm()
    margins = []
    convRates = []
    for k in range(0,len(pulledArm)):
        margins.append(marginsPerPrice[k][pulledArm[k]])
        convRates.append(conversionRateLevels[k][pulledArm[k]])

    price_configuration_margin = 0
    eval = GraphEvaluator(products_list=productList, click_prob_matrix=CLICK_PROB, lambda_prob=0.7, conversion_rates=convRates,
                    alphas=alphas, margins=margins, units_mean=units_mean, verbose=False)

    overall_margin = eval.computeMargin()
    print("Configuration {}; ConvRates {}; Margins {}; Overall Margin {}".format(pulledArm,convRates,margins,overall_margin))
    gLearner.update(overall_margin)


TypeError: object of type 'int' has no len()

In [20]:
import json
from Environment import Environment
from Model.Evaluator.GraphEvaluator import GraphEvaluator
from Model.Evaluator.Baseline import Baseline
from Model.Evaluator.OneStepEvaluator import OneStepEvaluator

from Model.Product import *
import numpy as np

marginsPerPrice = [
    [1, 2, 10, 16],
    [10, 20, 35, 40],
    [12, 15, 18, 21],
    [3, 8, 15, 21],
    [8, 10, 17, 26]
  ]

env = Environment(config_path='./Configs/config1.json')
arm = [1,0,2,1,3]
margins = [marginsPerPrice[i][arm[i]] for i in range(0,len(arm))]
print(margins)
n_experiments = 400
obtained_margins = []
env.setPriceLevels(arm)
for i in range(0,n_experiments):
  inters = env.round()
  total = 0
  for inter in inters:
    total += inter.linearizeMargin(marginsPerPrice)
    obtained_margins.append(inter.linearizeMargin(marginsPerPrice))
  total = total / len(inters)
  # obtained_margins.append(total)

print("Mean env ({} experiments) reward in a single iteration for config {}: {}".format(n_experiments, arm, np.array(obtained_margins).mean()))

config_path = "./Configs/config1.json"
if config_path != None:
  f = open(config_path)
  config = json.load(f)
  f.close()
  conf_classes = config["classes"]
  for uc in conf_classes:
      armConvRates = [uc["conversionRates"][i][arm[i]] for i in range(0,len(arm))]
      productList = [Product(int(key), uc["secondary"][key]) for key in uc["secondary"]]
      eval = GraphEvaluator(products_list=productList, click_prob_matrix=uc["clickProbability"], lambda_prob=uc["lambda"], conversion_rates=armConvRates,
                    alphas=uc["alphas"], margins=margins, units_mean=uc["unitsShape"], verbose=False)
      baseline = Baseline(products_list=productList, click_prob_matrix=uc["clickProbability"], lambda_prob=uc["lambda"], conversion_rates=armConvRates,
                    alphas=uc["alphas"], margins=margins, units_mean=uc["unitsShape"], verbose=False)
      oneStep = OneStepEvaluator(products_list=productList, click_prob_matrix=uc["clickProbability"], lambda_prob=uc["lambda"], conversion_rates=armConvRates,
                    alphas=uc["alphas"], margins=margins, units_mean=uc["unitsShape"], verbose=False)

print("Expected reward in a single iteration for config {}: {}".format(arm, eval.computeMargin()))
print("Baseline reward in a single iteration for config {}: {}".format(arm, baseline.computeMargin()))
print("One Step expected reward in a single iteration for config {}: {}".format(arm, oneStep.computeMargin()))



#inters[0].printInteractions()
#print([1,10,12,3,8])
#print(inters[0].linearizeBought())
#print(inters[0].linearizeNumUnits())
#print(inters[0].linearizeMargin(marginsPerPrice))

[2, 10, 18, 8, 26]
Mean env (400 experiments) reward in a single iteration for config [1, 0, 2, 1, 3]: 22.136730888750776
Expected reward in a single iteration for config [1, 0, 2, 1, 3]: 22.15
Baseline reward in a single iteration for config [1, 0, 2, 1, 3]: 7.7847
One Step expected reward in a single iteration for config [1, 0, 2, 1, 3]: 12.95


In [7]:
import json
from Environment import Environment
import numpy as np
from Learner.BruteForce import *
from Model.UserClass import *
from Model.Product import *
from Model.GraphProbabilities import *
from Model.Evaluator.GraphEvaluator import GraphEvaluator
from Model.Evaluator.Baseline import Baseline



f = open('./Configs/config3.json')
config = json.load(f)
f.close()
uc = config["classes"][0]

productList = [Product(int(key), uc["secondary"][key]) for key in uc["secondary"]]

conversionRateLevels = uc["conversionRates"]
marginsPerPrice = config["margins"]
click_prob = np.array(uc["clickProbability"])
lambda_p = uc["lambda"]
alphas = uc["alphas"]
units_mean = uc["unitsShape"]
num_prices = len(conversionRateLevels[0])
num_prods = len(alphas)

bf = BruteForce(num_prices=num_prices, num_products=num_prods)
optimal_arm = []

for i in range(0, num_prices**num_prods):
    pulledArm = bf.pull_arm()
    margins = []
    convRates = []
    for k in range(0,len(pulledArm)):
        margins.append(marginsPerPrice[k][pulledArm[k]])
        convRates.append(conversionRateLevels[k][pulledArm[k]])

    price_configuration_margin = 0
    eval = GraphEvaluator(products_list=productList, click_prob_matrix=click_prob, lambda_prob=lambda_p, conversion_rates=convRates,
                    alphas=alphas, margins=margins, units_mean=units_mean, verbose=False)
    eval2 = Baseline(products_list=productList, click_prob_matrix=click_prob, lambda_prob=lambda_p, conversion_rates=convRates,
                    alphas=alphas, margins=margins, units_mean=units_mean, verbose=False)

    overall_margin = eval.computeMargin()
    baseline = eval2.computeMargin()
    print("Configuration {}; ConvRates {}; Margins {}; Overall Margin {}; Baseline {}".format(pulledArm,convRates,margins,int(overall_margin*100)/100,int(baseline*100)/100))
    if overall_margin < baseline:
        print("VAFFANCULOOOOO")
    bf.update(overall_margin)

print(bf.get_optima())

Configuration [0, 0, 0, 0, 0]; ConvRates [0.9, 0.7, 0.6, 0.5, 0.8]; Margins [6, 6, 6, 6, 6]; Overall Margin 20.5; Baseline 4.67
Configuration [0, 0, 0, 0, 1]; ConvRates [0.9, 0.7, 0.6, 0.5, 0.7]; Margins [6, 6, 6, 6, 30]; Overall Margin 35.55; Baseline 5.13
Configuration [0, 0, 0, 0, 2]; ConvRates [0.9, 0.7, 0.6, 0.5, 0.2]; Margins [6, 6, 6, 6, 31]; Overall Margin 21.97; Baseline 4.71
Configuration [0, 0, 0, 0, 3]; ConvRates [0.9, 0.7, 0.6, 0.5, 0.05]; Margins [6, 6, 6, 6, 32]; Overall Margin 17.59; Baseline 4.58
Configuration [0, 0, 0, 1, 0]; ConvRates [0.9, 0.7, 0.6, 0.4, 0.8]; Margins [6, 6, 6, 30, 6]; Overall Margin 38.93; Baseline 5.24
Configuration [0, 0, 0, 1, 1]; ConvRates [0.9, 0.7, 0.6, 0.4, 0.7]; Margins [6, 6, 6, 30, 30]; Overall Margin 53.48; Baseline 5.7
Configuration [0, 0, 0, 1, 2]; ConvRates [0.9, 0.7, 0.6, 0.4, 0.2]; Margins [6, 6, 6, 30, 31]; Overall Margin 40.4; Baseline 5.28
Configuration [0, 0, 0, 1, 3]; ConvRates [0.9, 0.7, 0.6, 0.4, 0.05]; Margins [6, 6, 6, 30, 

In [ ]:
f = open('./Configs/config1.json')
config = json.load(f)
f.close()
uc = config["classes"][0]

productList = [Product(int(key), uc["secondary"][key]) for key in uc["secondary"]]

conversionRateLevels = np.array(uc["conversionRates"])
marginsPerPrice = np.array(config["margins"])
res = np.multiply(conversionRateLevels,marginsPerPrice)
print(res)
best_per_prod = []
for i in range(0,res.shape[0]):
    best_per_prod.append(np.max(res[i]))
best_per_prod = np.array(best_per_prod)
print("Best per product: {}".format(best_per_prod))
alphas = np.array(uc["alphas"])
print("Alphas: {}".format(alphas))
print(np.multiply(alphas, best_per_prod))
print(np.multiply(alphas, best_per_prod).sum())

In [ ]:
def numberToBase(n, b):
    if n == 0:
        return [0]
    digits = []
    while n:
        digits.append(int(n % b))
        n //= b
    return digits[::-1]

arr = numberToBase(1023,4)
remaining = [0 for i in range(0,5-len(arr))]
print(remaining + arr)

In [ ]:
np.multiply(np.multiply(np.array([0.2, 0.6, 0.9, 0.3, 0.45]),np.array([16, 35, 12, 21, 17])),np.array([0.3, 0.25, 0.15, 0.15, 0.15])).sum()

In [2]:
import json
from Environment import Environment
import numpy as np
from Learner.BruteForce import *
from Model.UserClass import *
from Model.Product import *
from Model.GraphProbabilities import *
from Model.Evaluator.GraphEvaluator import GraphEvaluator
from Model.Evaluator.Baseline import Baseline



f = open('./Configs/config1.json')
config = json.load(f)
f.close()
uc = config["classes"][0]

productList = [Product(int(key), uc["secondary"][key]) for key in uc["secondary"]]

conversionRateLevels = uc["conversionRates"]
marginsPerPrice = config["margins"]
click_prob = uc["clickProbability"]
lambda_p = uc["lambda"]
alphas = uc["alphas"]
num_prices = len(conversionRateLevels[0])
num_prods = len(alphas)

optimal_arm = []

pulledArm = [0, 3, 0, 2, 0] # [3, 2, 0, 3, 2] 
margins = []
convRates = []
for k in range(0,len(pulledArm)):
    margins.append(marginsPerPrice[k][pulledArm[k]])
    convRates.append(conversionRateLevels[k][pulledArm[k]])

price_configuration_margin = 0
eval = GraphEvaluator(products_list=productList, click_prob_matrix=click_prob, lambda_prob=lambda_p, conversion_rates=convRates,
                alphas=alphas, margins=margins, verbose=True)
eval2 = Baseline(products_list=productList, click_prob_matrix=click_prob, lambda_prob=lambda_p, conversion_rates=convRates,
                alphas=alphas, margins=margins, verbose=True)

overall_margin = eval.computeMargin()
baseline = eval2.computeMargin()
print("Configuration {}; ConvRates {}; Margins {}; Overall Margin {}; Baseline {}".format(pulledArm,convRates,margins,overall_margin,baseline))



AssertionError: 